## Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from pytorchtools import EarlyStopping

import numpy as np
import os
import pickle
import gc  # 메모리를 초기화 하는 모듈
from PIL import Image  #이미지 처리를 위한 모듈

from sklearn.metrics import accuracy_score, f1_score, log_loss


import random
import numpy as np
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

c:\Users\USER\anaconda3\envs\bax\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# GPU (cuda) 설정
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version: ', torch.__version__, ' Device: ', DEVICE)

Using PyTorch version:  1.13.1  Device:  cuda


In [4]:
# HTP 데이터프레임 로드
# House, Tree, Person 데이터를 각각 따로 불러와서 모델 훈련
with open(r"C:\BAX_HTP_project\deeplearning\df_metainfo.pkl","rb") as fr:
    person_data = pickle.load(fr)["person"]
person_full_folder = r'./full_images/' + person_data["id"].astype("str") + '.jpg'

In [5]:
# 정규화 작업 및 이미지 텐서화 후, np.stack 함수를 이용하여 1차원으로 만드는 과정
# 이미지 전처리 후, numpy 안의 stack 함수를 이용해 img 피처를 만든다
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
img_features = np.stack([preprocess(Image.open(img_path)) for img_path in person_full_folder]).astype("float32")

## Training

In [6]:
# 모델 설계

# pre-trained 된 ResNet101 1k 버전을 torch.hub를 통해 load
class HTP_Classifier(nn.Module):
    def __init__(self, n_output):
        super(HTP_Classifier, self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, n_output) 
        
    def forward(self, x):
        x = self.model(x)
        return x

# 데이터셋 레이블 배치에 맞게 합치기
class MyDataset(Dataset):
    def __init__(self, img_feature, label):
        self.data = img_feature 
        self.labels = label

    def __len__(self):
        return len(self.data)  # 데이터셋의 크기를 반환합니다.

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def training(model_htp, train_dataloader, test_dataloader):
    # model_htp = HTP_Classifier(n_output=class_counts)
    # model_htp.to(DEVICE)

    # 다중 분류
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_htp.parameters(), lr=1e-5)
    early_stopping = EarlyStopping(patience=5, verbose=True)

    for epoch in range(30):
        
        # train
        running_loss = 0.0
        for data in train_dataloader:
            model_htp.train()
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model_htp(inputs)
            loss = criterion(outputs, labels.to(torch.int64))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Train Loss ({label}): {running_loss / len(train_dataloader):.3f}')
        
        # test
        val_loss = 0.0
        for data in test_dataloader:
            model_htp.eval()
            with torch.no_grad():
                images, labels = data
                images = images.to(DEVICE)   
                labels = labels.to(DEVICE)
                outputs = model_htp(images)
                loss = criterion(outputs, labels.to(torch.int64))
                
                val_loss += loss.item()
        
        print(f'Val Loss ({label}): {val_loss / len(test_dataloader):.3f}')
        
        early_stopping(val_loss, model_htp)

        if early_stopping.early_stop:
            print(f'Early Stopping ({label})')
            torch.save(model_htp.state_dict(), PATH)
            break
    
    print('Finished Training')
    model_htp.load_state_dict(torch.load('checkpoint.pt'))

In [8]:
labels = ['eye_yn', 'leg_yn', 'mouth_yn', 'arm_yn']

for label in labels:
    PATH = f'./save_moel/ResNet101_{label}.pt'
    train_img_features = img_features[person_data[f'fold_{label}'] != 3]
    train_labels = person_data[person_data[f'fold_{label}'] != 3][label].reset_index(drop=True).values
    test_img_features = img_features[person_data[f'fold_{label}'] == 3]
    test_labels = person_data[person_data[f'fold_{label}'] == 3][label].reset_index(drop=True).values
    
    train_dataloader = DataLoader(MyDataset(train_img_features, train_labels), batch_size=4, shuffle=True)
    test_dataloader = DataLoader(MyDataset(test_img_features, test_labels), batch_size=4, shuffle=False)
    
    model_htp = HTP_Classifier(n_output=person_data[label].nunique())
    model_htp.to(DEVICE)

    training(model_htp, train_dataloader, test_dataloader)
    
    del model_htp, train_dataloader, test_dataloader
    torch.cuda.empty_cache()
    gc.collect()
    
    # break

Using cache found in C:\Users\USER/.cache\torch\hub\pytorch_vision_v0.10.0


Train Loss (eye_yn): 0.314
Val Loss (eye_yn): 0.156
Validation loss decreased (inf --> 7.787522).  Saving model ...
Train Loss (eye_yn): 0.126
Val Loss (eye_yn): 0.141
Validation loss decreased (7.787522 --> 7.047907).  Saving model ...
Train Loss (eye_yn): 0.069
Val Loss (eye_yn): 0.143
EarlyStopping counter: 1 out of 5
Train Loss (eye_yn): 0.027
Val Loss (eye_yn): 0.162
EarlyStopping counter: 2 out of 5
Train Loss (eye_yn): 0.020
Val Loss (eye_yn): 0.161
EarlyStopping counter: 3 out of 5
Train Loss (eye_yn): 0.013
Val Loss (eye_yn): 0.175
EarlyStopping counter: 4 out of 5
Train Loss (eye_yn): 0.007
Val Loss (eye_yn): 0.180
EarlyStopping counter: 5 out of 5
Early Stopping (eye_yn)
Finished Training


Using cache found in C:\Users\USER/.cache\torch\hub\pytorch_vision_v0.10.0


Train Loss (leg_yn): 0.254
Val Loss (leg_yn): 0.197
Validation loss decreased (inf --> 9.870692).  Saving model ...
Train Loss (leg_yn): 0.142
Val Loss (leg_yn): 0.179
Validation loss decreased (9.870692 --> 8.946040).  Saving model ...
Train Loss (leg_yn): 0.072
Val Loss (leg_yn): 0.168
Validation loss decreased (8.946040 --> 8.376342).  Saving model ...
Train Loss (leg_yn): 0.038
Val Loss (leg_yn): 0.198
EarlyStopping counter: 1 out of 5
Train Loss (leg_yn): 0.024
Val Loss (leg_yn): 0.199
EarlyStopping counter: 2 out of 5
Train Loss (leg_yn): 0.013
Val Loss (leg_yn): 0.220
EarlyStopping counter: 3 out of 5
Train Loss (leg_yn): 0.009
Val Loss (leg_yn): 0.184
EarlyStopping counter: 4 out of 5
Train Loss (leg_yn): 0.006
Val Loss (leg_yn): 0.190
EarlyStopping counter: 5 out of 5
Early Stopping (leg_yn)
Finished Training


Using cache found in C:\Users\USER/.cache\torch\hub\pytorch_vision_v0.10.0


Train Loss (mouth_yn): 0.358
Val Loss (mouth_yn): 0.279
Validation loss decreased (inf --> 13.936497).  Saving model ...
Train Loss (mouth_yn): 0.228
Val Loss (mouth_yn): 0.238
Validation loss decreased (13.936497 --> 11.908119).  Saving model ...
Train Loss (mouth_yn): 0.134
Val Loss (mouth_yn): 0.291
EarlyStopping counter: 1 out of 5
Train Loss (mouth_yn): 0.073
Val Loss (mouth_yn): 0.317
EarlyStopping counter: 2 out of 5
Train Loss (mouth_yn): 0.042
Val Loss (mouth_yn): 0.310
EarlyStopping counter: 3 out of 5
Train Loss (mouth_yn): 0.027
Val Loss (mouth_yn): 0.331
EarlyStopping counter: 4 out of 5
Train Loss (mouth_yn): 0.015
Val Loss (mouth_yn): 0.339
EarlyStopping counter: 5 out of 5
Early Stopping (mouth_yn)
Finished Training


Using cache found in C:\Users\USER/.cache\torch\hub\pytorch_vision_v0.10.0


Train Loss (arm_yn): 0.309
Val Loss (arm_yn): 0.224
Validation loss decreased (inf --> 11.222726).  Saving model ...
Train Loss (arm_yn): 0.175
Val Loss (arm_yn): 0.225
EarlyStopping counter: 1 out of 5
Train Loss (arm_yn): 0.112
Val Loss (arm_yn): 0.212
Validation loss decreased (11.222726 --> 10.576431).  Saving model ...
Train Loss (arm_yn): 0.063
Val Loss (arm_yn): 0.257
EarlyStopping counter: 1 out of 5
Train Loss (arm_yn): 0.029
Val Loss (arm_yn): 0.232
EarlyStopping counter: 2 out of 5
Train Loss (arm_yn): 0.019
Val Loss (arm_yn): 0.245
EarlyStopping counter: 3 out of 5
Train Loss (arm_yn): 0.012
Val Loss (arm_yn): 0.250
EarlyStopping counter: 4 out of 5
Train Loss (arm_yn): 0.008
Val Loss (arm_yn): 0.287
EarlyStopping counter: 5 out of 5
Early Stopping (arm_yn)
Finished Training
